In [1]:
import matplotlib as plt
import pandas as pd
import sklearn as sk

In [2]:
from pandas import read_csv
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

In [3]:
df = pd.read_csv("df_clean.csv",delimiter=',')

In [4]:
df = df.drop(columns=["Unnamed: 0","Average Amount/transaction/day","6_month_avg_chbk_amt"])

In [5]:
df = df.rename(columns={'Is declined':'is_declined','isForeignTransaction':'is_foreign_transaction','isHighRiskCountry':'is_high_risk_country'
                   ,'isFradulent':'is_fraudulent','Total Number of declines/day':'total_number_of_declines_per_day'})

In [6]:
df.shape

(3075, 8)

In [8]:
array = df.values
X = array[:,0:7]
y = array[:,7]

In [9]:
# Splitting the data into 60% training, 20 % Validation and 20 % Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [15]:
import imblearn
import collections


In [20]:
smt = SMOTE()

In [21]:
X_train,y_train = smt.fit_resample(X_train,y_train)

In [24]:
# Choose between different algorithms for the baseline

In [27]:
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', svm.SVC(gamma='auto')))
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LR: 0.964038 (0.013546)
KNN: 0.815142 (0.017462)
SVM: 0.688328 (0.021801)


In [29]:
# Logistic regression is the best model for the use case.
# Let's make a baseline

In [31]:
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_val)

C:\Users\dalil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
# Evaluate predictions
print(accuracy_score(y_val, predictions))
print(confusion_matrix(y_val, predictions))
print(classification_report(y_val, predictions))

0.9512195121951219
[[497  21]
 [  9  88]]
              precision    recall  f1-score   support

         0.0       0.98      0.96      0.97       518
         1.0       0.81      0.91      0.85        97

    accuracy                           0.95       615
   macro avg       0.89      0.93      0.91       615
weighted avg       0.95      0.95      0.95       615



In [33]:
tn, fp, fn, tp = confusion_matrix(y_val,predictions).ravel()

In [34]:
(tn, fp, fn, tp)

(497, 21, 9, 88)

In [35]:
# Tune hyperparameter with grid search

In [37]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['none','l1','l2','elasticnet']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.974238 using {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
0.974027 (0.007451) with: {'C': 100, 'penalty': 'none', 'solver': 'newton-cg'}
0.958044 (0.018252) with: {'C': 100, 'penalty': 'none', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'none', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'l1', 'solver': 'lbfgs'}
0.973922 (0.007551) with: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
0.973922 (0.007462) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.954259 (0.018782) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.887907 (0.024728) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'elasticnet', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'elasticnet', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'elasticnet', 'solver': 'li

In [39]:
model2 = LogisticRegression(C=1,penalty='l1',solver='liblinear')
model2.fit(X_train, y_train)
predictions2 = model2.predict(X_val)

In [40]:
print(accuracy_score(y_val, predictions2))
print(confusion_matrix(y_val, predictions2))
print(classification_report(y_val, predictions2))

0.9626016260162602
[[501  17]
 [  6  91]]
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98       518
         1.0       0.84      0.94      0.89        97

    accuracy                           0.96       615
   macro avg       0.92      0.95      0.93       615
weighted avg       0.97      0.96      0.96       615



In [41]:
model3= LogisticRegression(C=1,penalty='l1',solver='liblinear')
model3.fit(X_train, y_train)
predictions3 = model3.predict(X_test)

In [45]:
print(accuracy_score(y_test, predictions3))
print(confusion_matrix(y_test, predictions3))
print(classification_report(y_test, predictions3))

0.967479674796748
[[511  13]
 [  7  84]]
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98       524
         1.0       0.87      0.92      0.89        91

    accuracy                           0.97       615
   macro avg       0.93      0.95      0.94       615
weighted avg       0.97      0.97      0.97       615



In [43]:
# Lets reduce C to improve the power of regularization